<a href="https://colab.research.google.com/github/azycr4yy/customer-churn-prediction-pytorch/blob/main/CreditChurner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch import nn, optim
import pandas as pd

In [83]:
data = pd.read_csv("BankChurners.csv",index_col=False)

In [84]:
print(data.columns)

Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
      dtype='object')


In [85]:
data.drop(columns=['CLIENTNUM',
                   'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                  'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
          inplace=True)

In [86]:
print(data.columns)

Index(['Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'],
      dtype='object')


In [87]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore' , sparse_output = False).set_output(transform = 'pandas')
ohetransform = ohe.fit_transform(data[['Attrition_Flag','Gender','Education_Level','Marital_Status','Card_Category',"Income_Category"]])
data = pd.concat([data , ohetransform],axis=1).drop(columns=['Gender','Education_Level','Marital_Status','Card_Category','Attrition_Flag',"Income_Category"])

In [88]:
X = data.drop(columns=['Attrition_Flag_Existing Customer'])
y = data['Attrition_Flag_Existing Customer']

In [90]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)
y_test , y_train = y_test.to_numpy() , y_train.to_numpy()

In [102]:
#Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train.to('cpu'))
X_test = sc.transform(X_test.to('cpu'))

In [91]:
X_train , X_test , y_train , y_test = torch.tensor(X_train) , torch.tensor(X_test) , torch.tensor(y_train) , torch.tensor(y_test)
X_train , X_test , y_train , y_test = X_train.float() , X_test.float() , y_train.float() , y_test.float()

In [92]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

(torch.Size([8101, 38]),
 torch.Size([2026, 38]),
 torch.Size([8101]),
 torch.Size([2026]))

In [93]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X_train , X_test , y_train , y_test = X_train.to(device) , X_test.to(device) , y_train.to(device) , y_test.to(device)

In [94]:
class BinaryClassifier(nn.Module):
  def __init__(self, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.layer1 = nn.Linear(in_features=38,out_features=64)
    self.layer2 = nn.Linear(in_features=64,out_features=128)
    self.layer3 = nn.Linear(in_features=128,out_features=256)
    self.layer4 = nn.Linear(in_features=256,out_features=64)
    self.layer5 = nn.Linear(in_features=64,out_features=1)
    self.act = nn.ReLU()
  def forward(self,x)->torch.Tensor:
    x = self.layer1(x)
    x = self.act(x)
    x = self.layer2(x)
    x = self.act(x)
    x = self.layer3(x)
    x = self.act(x)
    x = self.layer4(x)
    x = self.act(x)
    x = self.layer5(x)
    return x

In [95]:
model_1 = BinaryClassifier().to(device)
model_1

BinaryClassifier(
  (layer1): Linear(in_features=38, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=256, bias=True)
  (layer4): Linear(in_features=256, out_features=64, bias=True)
  (layer5): Linear(in_features=64, out_features=1, bias=True)
  (act): Tanh()
)

In [96]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_1.parameters(),lr=0.001)

In [97]:
#AI did this No idea what happening lol
def accuracy_score(y_true, y_pred):
    # print(f"accuracy_score: y_true shape: {y_true.shape}, dtype: {y_true.dtype}")
    # print(f"accuracy_score: y_pred shape: {y_pred.shape}, dtype: {y_pred.dtype}")
    # # Print first few values
    # print(f"accuracy_score: y_true[:5]: {y_true[:5]}")
    # print(f"accuracy_score: y_pred[:5]: {y_pred[:5]}")


    # Ensure y_true has the same shape as y_pred for comparison
    original_y_true_shape = y_true.shape
    if y_true.shape != y_pred.shape:
        if y_true.dim() == 1 and y_pred.dim() == 2 and y_true.shape[0] == y_pred.shape[0]:
            y_true = y_true.unsqueeze(1)
            # print(f"accuracy_score: y_true unsqueezed to shape: {y_true.shape}")
        else:
            raise ValueError("Incompatible shapes for accuracy calculation")

    # Convert to consistent data type (e.g., long integers) for exact comparison
    y_true_int = y_true.long()
    y_pred_int = y_pred.long()
    # print(f"accuracy_score: y_true_int shape: {y_true_int.shape}, dtype: {y_true_int.dtype}")
    # print(f"accuracy_score: y_pred_int shape: {y_pred_int.shape}, dtype: {y_pred_int.dtype}")
    # # Print first few values after casting
    # print(f"accuracy_score: y_true_int[:5]: {y_true_int[:5]}")
    # print(f"accuracy_score: y_pred_int[:5]: {y_pred_int[:5]}")


    # Calculate the number of correct predictions
    comparison_result = (y_true_int == y_pred_int)
    # print(f"accuracy_score: comparison_result shape: {comparison_result.shape}, dtype: {comparison_result.dtype}")
    # print(f"accuracy_score: comparison_result[:5]: {comparison_result[:5]}")

    correct_predictions = comparison_result.sum().item()
    # print(f"accuracy_score: correct_predictions: {correct_predictions}")


    # Calculate accuracy
    total_samples = len(y_pred_int)
    # print(f"accuracy_score: total_samples (len(y_pred_int)): {total_samples}")
    acc = (correct_predictions / total_samples) * 100
    # print(f"accuracy_score: calculated acc: {acc}")


    # Revert y_true shape if it was unsqueezed (optional, for clarity if needed later)
    # y_true = y_true.view(original_y_true_shape)


    return acc

In [98]:
#training loop
epochs = 10000
for epoch in range(epochs):
  model_1.train()
  y_log = model_1(X_train)
  y_label = torch.sigmoid(y_log)
  loss = loss_fn(y_log,y_train.unsqueeze(1).float())
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  model_1.eval()
  with torch.inference_mode():
    y_test_log = model_1(X_test)
    y_test_label = torch.sigmoid(y_test_log)
    y_test_pred = torch.round(y_test_label)
    acc = accuracy_score(y_test,y_test_pred)
  if epoch%1000==0: # Changed back to 1000 for less verbose output during diagnosis
    print(f"Epoch: {epoch} | Loss: {loss.item()} | Accuracy: {acc}")

Epoch: 0 | Loss: 0.6855131983757019 | Accuracy: 83.85982230997038
Epoch: 1000 | Loss: 0.0001067182602128014 | Accuracy: 100.0
Epoch: 2000 | Loss: 3.212456431356259e-05 | Accuracy: 100.0
Epoch: 3000 | Loss: 1.4268033737607766e-05 | Accuracy: 100.0
Epoch: 4000 | Loss: 7.322742021642625e-06 | Accuracy: 100.0
Epoch: 5000 | Loss: 4.12806230087881e-06 | Accuracy: 100.0
Epoch: 6000 | Loss: 2.392238229731447e-06 | Accuracy: 100.0
Epoch: 7000 | Loss: 1.4417646525544114e-06 | Accuracy: 100.0
Epoch: 8000 | Loss: 8.293309861073794e-07 | Accuracy: 100.0
Epoch: 9000 | Loss: 5.591296599050111e-07 | Accuracy: 100.0


In [100]:
model_1.eval()
with torch.inference_mode():
    y_final_logits = model_1(X_test)
    y_final_probs = torch.sigmoid(y_final_logits)
    y_final_preds = torch.round(y_final_probs)
from sklearn.metrics import confusion_matrix, classification_report
y_true = y_test.cpu().numpy()
y_pred = y_final_preds.cpu().numpy()
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))


Confusion Matrix:
 [[ 327    0]
 [   0 1699]]

Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       327
         1.0       1.00      1.00      1.00      1699

    accuracy                           1.00      2026
   macro avg       1.00      1.00      1.00      2026
weighted avg       1.00      1.00      1.00      2026

